### Conectamos Apache Kafka con Spark Structured streaming

In [0]:
# read the tweet data from socket
tweet_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "127.0.0.1:9092") \
    .option("subscribe", "test") \
    .load()

# type cast the column value
tweet_df_string = tweet_df.selectExpr("CAST(value AS STRING)")

In [0]:
tweet_df_string

Out[2]: DataFrame[value: string]

Se dividen las palabras por espacio, filtrando por hashtag (#) y posteriormente se agrupan.

In [0]:
from pyspark.sql.functions import explode,split,col

tweets_tab = tweet_df_string.withColumn('word', explode(split(col('value'), ' '))) \
    .groupBy('word') \
    .count() \
    .sort('count', ascending=False). \
    filter(col('word').contains('#'))

In [0]:
tweets_tab

Out[4]: DataFrame[word: string, count: bigint]

In [0]:
writeTweet = tweets_tab.writeStream. \
    outputMode("complete"). \
    format("memory"). \
    queryName("tweetquery"). \
    trigger(processingTime='2 seconds'). \
    start()

In [0]:
%sql 
select *  
from tweetquery
limit 100

word,count
#Ucrania,61
#Rusia,29
"#Ucrania,",8
#Azovstal,5
#Mariupol,5
#Ucrania.,5
#ÚLTIMAHORA‼️,5
#3May,5
#URGENTE‼️ 🇷🇺,5
#Kiev,4
